# Логистическая регрессия

###### Импорт необходимых библиотек

In [91]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import sys

###### Импорт датасета

In [5]:
df = pd.read_csv('data-logistic.csv', header=None)

In [7]:
df.head(5)

,0,1,2
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025
2,-1,-1.247395,0.749425
3,1,2.309374,1.899836
4,1,0.849143,2.407750


In [12]:
# разбиение на признаки и целевую переменную
X = df[[1, 2]]
y = df[0].astype(int)

###### Написание логистической регрессии 

In [164]:
# функция лог. регрессии с использованием l2 регуляризации (переменная С):
def l2_log_regression(X, y, k, w, C, epsilon, max_iter):
    w1 = w[0]
    w2 = w[1]
    for iteration in range(max_iter):
        summa1 = summa2 = 0
        for i in range(len(X)):
            summa = 1 - 1 / (1 + np.exp(-y[i] * (w1 * X[1][i] + w2 * X[2][i])))
            summa1 += np.sum(y[i] * X[1][i] * summa)
            summa2 += np.sum(y[i] * X[2][i] * summa)
        w1_n = w1 + k / len(X) * summa1 - k * C * w1
        w2_n = w2 + k / len(X) * summa2 - k * C * w2
        if (np.sqrt((w1_n - w1) ** 2 + (w2_n - w2) ** 2) < epsilon).all():
            n = iteration # - номер итерации
            break
        w1 = w1_n
        w2 = w2_n
        n = iteration
        
    predictions = []
    for pred in range(len(y)):
        predictions.append(1 / (1 + np.exp(np.dot(-w1, X[1][pred]) + np.dot(-w2, X[2][pred]))))
    return n + 1, predictions

In [165]:
# практическая реализация функции лог. регрессии для датасета
iteration_normal, predicts_normal = l2_log_regression(X, y, 0.1, np.array([0, 0]), 0, 0.00001, 10000)
# с регуляризацией
iteration_reg, predicts_reg = l2_log_regression(X, y, 0.1, np.array([0, 0]), 10, 0.00001, 10000)

###### Проверка логистической регрессии с использованием roc_auc_score

In [166]:
print('roc_auc_score без регуляризации -', round(roc_auc_score(y, predicts_normal), 3))
print(f'Сходимость достигнута на {iteration_normal} итерации')
print('roc_auc_score с регуляризацией -', round(roc_auc_score(y, predicts_reg), 3))
print(f'Сходимость достигнута на {iteration_reg} итерации')

roc_auc_score без регуляризации - 0.927
Сходимость достигнута на 244 итерации
roc_auc_score с регуляризацией - 0.936
Сходимость достигнута на 8 итерации


Как видно из подсчитанных оценок качества, величина незначительно увеличилась, в то же время, сходимость была достигнута за гораздо меньшее число итераций при использовании регуляризации $l2$.